In [34]:
import numpy as np
import pandas as pd
DATASET_PATH='data/housing.csv'
PREPARED_DATASET_PATH = 'data/housing_prepared.csv'

In [26]:
class DataPipeline:
   #Загрузка дата фрейма
    def __init__(self,DATASET_PATH):
        self.df=pd.read_csv(DATASET_PATH)
        self.medians={}
        
   #Посчитаем медианные значения для столбцов из списка     
    def calculation_medians(self,list_columns):
        for i in list_columns:
            self.medians[i]=self.df[i].median()
            
    #Заполним пустые занчения в полях в списке
    def filling_void_values (self,list_change):
        for i in list_change:
            self.df[i].fillna(self.medians[i],inplace=True)
            
    #Проверка географических координат
    def geo_check(self):
        #меняем нулевые значения долготы
        self.df.loc[self.df['longitude']==0,'longitude']=self.medians['longitude']
        #Меняем ошибочно веведенныеположительные значения
        self.df.loc[self.df['longitude']>0,'longitude']=self.df.loc[self.df['longitude']>0,'longitude']*-1
        #Проверка координат на приналлежность Гео Объекту
        self.df.loc[~self.df['longitude'].between(-126,-100),'longitude']=self.medians['longitude']
        #Проверки по широте
        self.df.loc[self.df['latitude']==0,'latitude']=self.medians['latitude']
        self.df.loc[self.df['latitude']<0,'latitude']=self.df.loc[self.df['latitude']<0,'latitude']*-1
        self.df.loc[~self.df['latitude'].between(20,45),'latitude']=self.medians['latitude']
     
    #Новые фичи
    def features(self):
         # Доля спален в общем кол-ве комнат
        self.df['bedroom_share'] = self.df['total_bedrooms'] / self.df['total_rooms'] * 100

        # Сколько человек в среднем живут в одной комнате
        self.df['population_per_room'] = self.df['population'] / self.df['total_rooms']
        
        # Обработка категорий
        self.df = pd.concat([self.df, pd.get_dummies(self.df['ocean_proximity'])], axis=1)




In [27]:
data=DataPipeline(DATASET_PATH)
list_columns=['longitude','latitude','housing_median_age','total_bedrooms','population']
data.calculation_medians(list_columns)

In [28]:
list_change=['housing_median_age','total_bedrooms','population']
data.filling_void_values(list_change)

In [29]:
data.geo_check()

In [30]:
data.features()

In [32]:
data.df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,id,bedroom_share,population_per_room,-,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,0,14.659091,0.365909,0,0,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,1,15.579659,0.338217,0,0,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,2,12.951602,0.338105,0,0,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,3,18.445840,0.437991,0,0,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,4,17.209588,0.347265,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND,20635,22.462462,0.507508,0,0,1,0,0,0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND,20636,21.520803,0.510760,0,0,1,0,0,0
20637,-121.22,39.43,17.0,2254.0,485.0,1165.0,433.0,1.7000,92300.0,INLAND,20637,21.517303,0.516859,0,0,1,0,0,0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND,20638,21.989247,0.398387,0,0,1,0,0,0


In [35]:
data.df.to_csv(PREPARED_DATASET_PATH,encoding='utf-8')